In [61]:
import pandas as pd
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras
import time

In [62]:
df=pd.read_csv('twiit.csv')

In [63]:
df

,created_at,source,original_text,polarity,subjectivity,lang,favourite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,Wed Aug 03 20:19:13 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",#Pelosi airplane landed safely in #Taiwan,0.500000,0.500000,en,0,0,DzCritical,318,373,NaN,"[{'text': 'Pelosi', 'indices': [0, 7]}, {'text...",[],Algerie
1,Tue Aug 02 15:24:42 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Watch the video of the beginning of the Chines...,0.000000,0.000000,en,0,0,toopsat,764,144,False,"[{'text': 'Pelosi', 'indices': [149, 156]}, {'...",[],NaN
2,Tue Aug 02 15:02:35 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",#Pelosi \n#Taipei \n#taiwan\n#XiJinping \n#Chi...,0.000000,0.000000,en,0,0,NassimaLilEmy,64,47,False,"[{'text': 'Pelosi', 'indices': [0, 7]}, {'text...",[],Algerie
3,Mon Aug 01 13:51:42 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",#HOBIPALOOZA #LaAcademiaExpulsion #WEURO2022 #...,0.433333,0.733333,en,0,0,d_dhayae,60,463,False,"[{'text': 'HOBIPALOOZA', 'indices': [0, 12]}, ...",[],Chlef
4,Sun Jul 31 20:02:20 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",#Pelosi\n#china\nChina Time ✌️ https://t.co/tE...,0.000000,0.000000,en,0,0,Mohamme65404115,39,206,False,"[{'text': 'Pelosi', 'indices': [0, 7]}, {'text...",[],Algerie
5,Sun Jul 31 13:22:22 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",V382 Intro Logo Animation Blue Smoke 💙\n\nIf y...,0.169841,0.350397,en,0,0,Yacine_Kelala8,350,1,False,"[{'text': 'YK', 'indices': [261, 264]}, {'text...",[],États-Unis
6,Sun Aug 07 05:55:54 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","TWINNING FOR ""AFRO IN SPACE""🛰️\n""SENORITA"" COU...",0.000000,0.000000,en,0,0,drax_africa,559,1280,False,"[{'text': 'fashion', 'indices': [113, 121]}, {...",[],Zambia
7,Thu Aug 04 01:42:24 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",When you're missing her but she's far.\n#Taiwa...,-0.050000,0.525000,en,0,0,MukukaReal,119,179,False,"[{'text': 'Taiwan', 'indices': [39, 46]}, {'te...",[],Ndola
8,Wed Aug 03 17:05:47 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...","🤔\n\n#China's Mortgage Crisis, Banks are Faili...",0.000000,0.000000,en,0,0,derPaddy,678,183,False,"[{'text': 'China', 'indices': [3, 9]}, {'text'...",[],:D


In [64]:
df.info

<bound method DataFrame.info of                        created_at  \
0  Wed Aug 03 20:19:13 +0000 2022   
1  Tue Aug 02 15:24:42 +0000 2022   
2  Tue Aug 02 15:02:35 +0000 2022   
3  Mon Aug 01 13:51:42 +0000 2022   
4  Sun Jul 31 20:02:20 +0000 2022   
5  Sun Jul 31 13:22:22 +0000 2022   
6  Sun Aug 07 05:55:54 +0000 2022   
7  Thu Aug 04 01:42:24 +0000 2022   
8  Wed Aug 03 17:05:47 +0000 2022   

                                              source  \
0  <a href="http://twitter.com/download/android" ...   
1  <a href="https://mobile.twitter.com" rel="nofo...   
2  <a href="http://twitter.com/download/android" ...   
3  <a href="http://twitter.com/download/android" ...   
4  <a href="http://twitter.com/download/android" ...   
5  <a href="http://twitter.com/download/android" ...   
6  <a href="http://twitter.com/download/android" ...   
7  <a href="http://twitter.com/download/android" ...   
8  <a href="https://mobile.twitter.com" rel="nofo...   

                                    

In [65]:
df.columns

Index(['created_at', 'source', 'original_text', 'polarity', 'subjectivity',
       'lang', 'favourite_count', 'retweet_count', 'original_author',
       'followers_count', 'friends_count', 'possibly_sensitive', 'hashtags',
       'user_mentions', 'place'],
      dtype='object')

In [66]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
polarity,9.0,0.117019,0.207931,-0.05,0.0,0.0,0.169841,0.500000
subjectivity,9.0,0.234303,0.294130,0.00,0.0,0.0,0.500000,0.733333
favourite_count,9.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000
retweet_count,9.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000
followers_count,9.0,327.888889,281.794005,39.00,64.0,318.0,559.000000,764.000000
friends_count,9.0,319.555556,387.905952,1.00,144.0,183.0,373.000000,1280.000000


In [76]:
conn=psycopg2.connect(database='twitter-analysis',user='postgres',host='localhost',port='5432',password='1234')

In [77]:
cursor=conn.cursor()
conn.autocommit=True

In [78]:
print("Database Connected Sucsussfully")

Database Connected Sucsussfully


In [79]:
cursor.execute("DROP TABLE IF EXISTS Twitter")

In [80]:
df.dtypes

created_at             object
source                 object
original_text          object
polarity              float64
subjectivity          float64
lang                   object
favourite_count         int64
retweet_count           int64
original_author        object
followers_count         int64
friends_count           int64
possibly_sensitive     object
hashtags               object
user_mentions          object
place                  object
dtype: object

In [81]:
#Creating Tables
sql='''CREATE Table Twitter 
(created_at text NOT NULL,
source text,
original_text text,
polarity int,
subjectivity int,
lang text,
favourite_count int,
retweet_count int,
original_author text,
followers_count int,
friends_count int,
possibly_sensitive text,
hashtags text,
user_mentions text,
place text)'''

In [82]:
cursor.execute(sql)

In [83]:
print("Tabel Created Sucessfully")

Tabel Created Sucessfully


In [84]:
insert_query_execute_val = f"""insert into Twitter(created_at,source,original_text,polarity,subjectivity,lang,favourite_count
,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
                    ) 
                    values %s """
                    
psycopg2.extras.execute_values(cursor, insert_query_execute_val, df.values)

In [85]:
print("CSV Loaded To Tabe SUcussfully")

CSV Loaded To Tabe SUcussfully
